<a href="https://colab.research.google.com/github/Pelmenoff/data_science/blob/main/hw10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Завантаження даних
(x, y), _ = fashion_mnist.load_data()

# Спочатку розділяємо дані на 70% тренувальні і 30% для тестування та валідації
x_train, x_test_raw, y_train, y_test_raw = train_test_split(x, y, train_size=0.7, random_state=555)

# Тепер розділимо ці 30% на 20% для тесту і 15% для валідації
# 20% від загального числа = 20/30 частини від x_test_raw та y_test_raw
test_size = 20 / 30
x_val, x_test, y_val, y_test = train_test_split(x_test_raw, y_test_raw, test_size=test_size, random_state=555)

print("Train data size", x_train.shape)
print("Test data size", x_test.shape)
print("Validation data size", x_val.shape)

# Масштабування зображень до діапазону [0, 1]
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Розширення розмірності (додавання каналу)
x_train = np.expand_dims(x_train, -1)
x_val = np.expand_dims(x_val, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot кодування міток
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

Train data size (42000, 28, 28)
Test data size (12000, 28, 28)
Validation data size (6000, 28, 28)


In [ ]:
model = Sequential()

# Перший згортковий шар
model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))

# Другий згортковий шар
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Перетворення даних в одновимірний вектор
model.add(Flatten())

# Перший повнозв'язний шар
model.add(Dense(256, activation='relu'))

# Вихідний шар
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 128)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 256)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 256)               1

In [ ]:
# Компільовання моделі
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [ ]:
# Тренування моделі
history = model.fit(x_train, y_train, batch_size=64,
                    epochs=50, validation_data=(x_val, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/50
657/657 [==============================] - 231s 349ms/step - loss: 0.4644 - accuracy: 0.8297 - val_loss: 0.3239 - val_accuracy: 0.8797 - lr: 0.0010
Epoch 2/50
657/657 [==============================] - 224s 340ms/step - loss: 0.3042 - accuracy: 0.8871 - val_loss: 0.2789 - val_accuracy: 0.8962 - lr: 0.0010
Epoch 3/50
657/657 [==============================] - 223s 339ms/step - loss: 0.2489 - accuracy: 0.9075 - val_loss: 0.2921 - val_accuracy: 0.8943 - lr: 0.0010
Epoch 4/50
657/657 [==============================] - 223s 339ms/step - loss: 0.2129 - accuracy: 0.9208 - val_loss: 0.2511 - val_accuracy: 0.9105 - lr: 0.0010
Epoch 5/50
657/657 [==============================] - 223s 340ms/step - loss: 0.1793 - accuracy: 0.9325 - val_loss: 0.2337 - val_accuracy: 0.9150 - lr: 0.0010
Epoch 6/50
657/657 [==============================] - 218s 332ms/step - loss: 0.1520 - accuracy: 0.9421 - val_loss: 0.2555 - val_accuracy: 0.9107 - lr: 0.0010
Epoch 7/50
657/657 [==========================

In [ ]:
# Оцінка моделі
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}, accuracy: {test_acc}')

375/375 [==============================] - 17s 46ms/step - loss: 0.2390 - accuracy: 0.9155
Test loss: 0.23901939392089844, accuracy: 0.9154999852180481
